In [1]:
from src.component.data_info import *
from datetime import datetime, timedelta

In [2]:
# specify the date we want to fetch our data
from datetime import datetime, timedelta

import pandas as pd
#specify the current time
current_date = pd.to_datetime(datetime.utcnow(),utc=True).floor('h')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=90)#it should only fetch for the last hour but because we dont want the pipline to break in case this data or the provois our was not availble so thats why we added for the entier month
fetch_data_from = pd.Timestamp(year=fetch_data_from.year, month=fetch_data_from.month, day=1,hour=5,tz='UTC')
print(fetch_data_to, fetch_data_from )

current_date=Timestamp('2025-02-03 00:00:00+0000', tz='UTC')
2025-02-03 00:00:00+00:00 2024-11-01 05:00:00+00:00


C:\Users\Hadeel\AppData\Local\Temp\ipykernel_39608\3565368181.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = pd.to_datetime(datetime.utcnow(),utc=True).floor('h')


In [3]:
#fetch batch of data from the wherehous

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    demand_values = load_full_data(from_date_,to_date_)
    demand_values = demand_values[(demand_values.date >= from_date_) & (demand_values.date < to_date_)]


    # shift the data to pretend this is recent data
    demand_values['date'] += timedelta(days=7*52)

    demand_values.sort_values(by=['sub_region_code', 'date'], inplace=True)

    return demand_values

In [4]:
data=fetch_batch_raw_data(fetch_data_from,fetch_data_to)

from_date=Timestamp('2024-11-01 05:00:00+0000', tz='UTC'), to_date_=Timestamp('2024-02-05 00:00:00+0000', tz='UTC')
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-03.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-04.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-05.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-06.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-07.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-08.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-09.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-11-10.json
Loading file D:\Electricity_demand_p

In [5]:
# string to datetime
data['date'] = pd.to_datetime(data['date'], utc=True)

# add column with Unix epoch milliseconds
data['seconds'] = data['date'].astype('int64') // 10**6

In [6]:
data.isna().sum()

date               0
sub_region_code    0
demand             0
temperature_2m     0
seconds            0
dtype: int64

In [7]:
data

,date,sub_region_code,demand,temperature_2m,seconds
220,2024-11-01 05:00:00+00:00,0,1559,8.5085,1730437200000
209,2024-11-01 06:00:00+00:00,0,1528,7.9085,1730440800000
198,2024-11-01 07:00:00+00:00,0,1512,7.9085,1730444400000
187,2024-11-01 08:00:00+00:00,0,1520,8.9585,1730448000000
176,2024-11-01 09:00:00+00:00,0,1524,9.5585,1730451600000
...,...,...,...,...,...
25585,2025-02-02 19:00:00+00:00,10,1706,7.3085,1738522800000
25574,2025-02-02 20:00:00+00:00,10,1780,7.1585,1738526400000
25563,2025-02-02 21:00:00+00:00,10,1926,7.1085,1738530000000
25552,2025-02-02 22:00:00+00:00,10,2147,7.2585,1738533600000


In [8]:
import hopsworks
from src.component import feature_group_config

d:\poetry_virtualenvs\src-o1faXYQI-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
hopsworks.__version__

'4.1.5'

In [10]:
project = hopsworks.login(
    project=feature_group_config.HOPSWORKS_PROJECT_NAME,
    api_key_value=feature_group_config.HOPSWORKS_API_KEY
)
feature_store = project.get_feature_store()

FEATURE_GROUP_NAME = 'electricity_demand_feature_group'
FEATURE_GROUP_VERSION = 3

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['sub_region_code','date'],
    event_time='date',
)

2025-02-03 00:02:32,239 INFO: Initializing external client
2025-02-03 00:02:32,240 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-03 00:02:33,445 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212591


In [11]:
feature_group.insert(data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 25773/25773 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: electricity_demand_feature_group_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212591/jobs/named/electricity_demand_feature_group_3_offline_fg_materialization/executions


(Job('electricity_demand_feature_group_3_offline_fg_materialization', 'SPARK'),
 None)